In [1]:
using Pkg
Pkg.activate("ScriptEnvironment"; shared=true)
Pkg.add(Pkg.PackageSpec(; path="/users/fergalbyrne/src/ogmajl/OgmaJL"))
using OgmaJL

 Activating environment at `~/.juliapro/JuliaPro_v1.4.2-1/environments/ScriptEnvironment/Project.toml`


   Updating git-repo `/Users/fergalbyrne/src/ogmajl/OgmaJL`


    Fetching: [======================>                  ]  

Fetching: [========================================>]  100.0 %

   Updating registry at `~/.juliapro/JuliaPro_v1.4.2-1/registries/JuliaPro`
######################################################################### 100.0%
  Resolving package versions...
   Updating `~/.juliapro/JuliaPro_v1.4.2-1/environments/ScriptEnvironment/Project.toml`
  [1d1ac247] ↑ OgmaJL v0.2.1 #master (/Users/fergalbyrne/src/ogmajl/OgmaJL) ⇒ v0.3.1 #master (/Users/fergalbyrne/src/ogmajl/OgmaJL)
   Updating `~/.juliapro/JuliaPro_v1.4.2-1/environments/ScriptEnvironment/Manifest.toml`
  [1d1ac247] ↑ OgmaJL v0.2.1 #master (/Users/fergalbyrne/src/ogmajl/OgmaJL) ⇒ v0.3.1 #master (/Users/fergalbyrne/src/ogmajl/OgmaJL)
┌ Info: Precompiling OgmaJL [1d1ac247-29ad-408a-a8db-7f943c355b0e]
└ @ Base loading.jl:1260


In [2]:
import Random

In [3]:
struct Hierarchy 
    inputSizes
    hiddenSizes
    radii
    tickLengths
    encoders
    decoders
end

In [4]:
function make_encoder(i, inputSizes, hiddenSizes, radii, tickLengths)
    if i == 1
        SparseFocalEncoder(inputSizes, hiddenSizes[i], radii[i])
    else
        ins = fill(hiddenSizes[i - 1], tickLengths[i])
        SparseFocalEncoder(ins, hiddenSizes[i], radii[i])
    end
end

make_encoder (generic function with 1 method)

In [5]:
function make_predictors(i, inputSizes, hiddenSizes, radii, tickLengths)
    if i == 1
        [SparseFocalEncoder(fill(hiddenSizes[i], 2), inputSize, radii[i]) for inputSize in inputSizes]
    else
        outs = fill(hiddenSizes[i - 1], tickLengths[i])
        [SparseFocalEncoder(fill(hiddenSizes[i], 2), out, radii[i]) for out in outs]
    end
end

make_predictors (generic function with 1 method)

In [6]:
function Hierarchy(inputSizes, hiddenSizes, radii, tickLengths)
    encoders = [make_encoder(i, inputSizes, hiddenSizes, radii, tickLengths) for i in 1:length(hiddenSizes)]
    decoders = [make_predictors(i, inputSizes, hiddenSizes, radii, tickLengths) for i in 1:length(hiddenSizes)]
    Hierarchy(inputSizes, hiddenSizes, radii, tickLengths, encoders, decoders)
end

Hierarchy

In [7]:
inSizes = [XIO3(3, 4, 3), XIO3(5, 3, 2)]
hidSizes = [XIO3(5, 5, 3), XIO3(4, 4, 2), XIO3(3, 3, 1)]
radii = [[1, 1], [2, 2], [1, 1]]
tickLengths = [1, 2, 2]

3-element Array{Int64,1}:
 1
 2
 2

In [8]:
Random.seed!(34567)
h = Hierarchy(inSizes, hidSizes, radii, tickLengths)
length(h.encoders)

3

In [9]:
println(h.encoders[3].weight_tensors[2].ws)
println(h.decoders[3][1].weight_tensors[2].ws[1:20])

Float32[0.5645666, 0.15309635, 0.7359637, 0.71544, 0.799425, 0.63843274, 0.016764954, 0.34932443, 0.7868037, 0.5357577, 0.30784723, 0.39956483, 0.22885388, 0.708205, 0.31170255, 0.016708238, 0.31010646, 0.946049, 0.9716537, 0.21447033, 0.89573425, 0.038455434, 0.1067517, 0.31365073, 0.68066305, 0.21302518, 0.074583754, 0.3654082, 0.6081509, 0.27897733, 0.51189524, 0.066890344, 0.28793395, 0.8672033, 0.60227334, 0.18269815, 0.8613341, 0.89417404, 0.67564845, 0.24426123, 0.80081415, 0.25180587, 0.68101585, 0.8705161, 0.59676754, 0.6805112, 0.59569776, 0.3241796, 0.08708398, 0.13321671, 0.37594044, 0.94716084, 0.19471933, 0.9137607, 0.8102125, 0.4411426, 0.48457745, 0.07716258, 0.07737787, 0.47258377, 0.12880258, 0.33035645, 0.8233179, 0.27221224, 0.7047646, 0.3407676, 0.9115499, 0.12632395, 0.80582714, 0.30223784, 0.8823254, 0.10311093, 0.2997108, 0.3291588, 0.796909, 0.078478836, 0.7173148, 0.08376813, 0.28918144, 0.007468909, 0.3401631, 0.8745441, 0.79374135, 0.6804792, 0.15814999, 0.2

In [10]:
h.encoders[3].inputs[1] .= rand_array(AW, h.hiddenSizes[2])
h.encoders[3].inputs[2] .= rand_array(AW, h.hiddenSizes[2])
h.encoders[3].outputs .= zero_array(AW, h.hiddenSizes[3])

9-element Array{Float32,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [11]:
forward!(h.encoders[3])

9-element Array{Float32,1}:
 -2.375828
 -1.7389026
  0.048776746
 -0.7783632
 -1.2332242
 -0.8237661
 -0.42653063
 -1.1708118
 -0.38157976

In [12]:
import OgmaJL.forward!

In [13]:
function forward!(m::FocalMatrix, ins::Vector{AW}, outs::Vector{AW})
    idxW = 1
    local diz = m.isize.z
    local dix = 1; local diy = 1; local o = 0.0
    osize = m.osize
    isize = m.isize
    ws = m.ws
    xs, ys = m.xs, m.ys
    local bz = isize.z
    for ox::XIO in 1:osize.x
        ax, bx = xs[ox, :]
        dix = 1 + bx - ax
        for oy::XIO in 1:osize.y
            ay, by = ys[oy, :]
            diy = 1 + by - ay
            for oz::XIO in 1:osize.z
                oPos = oz + osize.z * (oy - 1) + (osize.z * osize.y) * (ox - 1)
                o = outs[oPos]
                for ix::XIO in ax:bx, iy::XIO in ay:by
                    iPos = isize.z * (iy - 1) + (isize.z * isize.y) * (ix - 1)
                    for iz::XIO in 1:bz
                        o = o + ins[iPos + iz] * ws[idxW + iz]
                    end
                    idxW += diz
                end
                outs[oPos] = o
            end
        end
    end
    outs
end

forward! (generic function with 4 methods)

In [14]:
function forward!(e::SparseFocalEncoder)
    for i in 1:length(e.weight_tensors)
        e.outputs .= forward!(e.weight_tensors[i], e.inputs[i], e.outputs)
        println(i, e.outputs)
    end
    e.outputs
end

forward! (generic function with 4 methods)

In [15]:
function forward!(h::Hierarchy, ins::Vector{Vector{AW}})
    h.encoders[1].inputs .= ins
    for e in 1:length(h.encoders)
        if e > 1
            for j in 2:length(h.encoders[e].inputs)
                h.encoders[e].inputs[j] .= h.encoders[e].inputs[j - 1]
            end
            h.encoders[e].inputs[1] .= h.encoders[e - 1].outputs
        end
        h.encoders[e].outputs .= 0.0
        forward!(h.encoders[e])  
        println("level $e:\ninputs:\n", h.encoders[e].inputs, "\noutputs:\n", h.encoders[e].outputs)
    end
end

forward! (generic function with 5 methods)

In [16]:
function reset!(h::Hierarchy)
    for enc in h.encoders
        for input in enc.inputs
            input .= 0.0 
        end
        enc.outputs .= 0.0
    end
end

reset! (generic function with 1 method)

In [17]:
Random.seed!(89123)
start_ins = [rand(AW, length(input)) for input in h.encoders[1].inputs]
reset!(h)

In [18]:
forward!(h, start_ins)

1Float32[2.7341948, 3.7357752, 3.9471684, 5.5391846, 4.7489758, 5.7991548, 6.1687202, 5.486372, 5.2629476, 6.1053886, 6.8938465, 7.705325, 3.644525, 3.155771, 3.2103758, 5.9293637, 5.388917, 6.1502347, 8.454125, 7.864378, 9.3291235, 6.9130926, 8.9074745, 8.517097, 7.5743847, 6.847526, 7.9313703, 6.2211356, 6.4321346, 6.701304, 5.9351974, 4.389, 6.8345246, 7.775779, 7.205558, 8.822968, 8.870237, 7.145187, 7.471397, 8.377363, 9.385347, 8.75828, 5.9227076, 4.884611, 4.198777, 3.570382, 3.2359755, 5.473711, 6.12904, 6.3310866, 4.4572926, 4.8412547, 5.449081, 5.5652947, 4.685097, 5.126682, 4.103604, 3.4805522, 2.9772797, 3.014519, 3.2823505, 3.999083, 3.5933135, 4.229941, 5.3103304, 7.88375, 5.1132307, 5.678768, 5.725518, 4.8154416, 5.0579963, 4.762769, 3.0144837, 2.670958, 2.2895691]
2Float32[5.023918, 5.3874598, 4.8634744, 8.309584, 7.992849, 8.434514, 9.472625, 8.4973755, 8.118412, 7.7828946, 9.49241, 10.899217, 5.970661, 5.4828725, 5.812313, 9.21267, 8.155136, 8.372528, 12.946195, 11.91

In [19]:
println(h.encoders[end].outputs)

Float32[628.0311, 1125.0378, 1042.0151, 1073.0894, 1645.9191, 820.4836, 610.6897, 1240.5907, 432.57755]


In [20]:
forward!(h.encoders[2])

1Float32[329.78833, 230.77928, 325.4293, 372.52838, 333.68506, 355.2007, 228.98228, 251.99353, 364.22644, 384.64645, 451.16382, 418.54965, 467.9811, 445.20676, 336.06516, 289.43478, 346.6008, 325.68512, 416.63342, 516.20215, 391.5713, 442.821, 279.20056, 299.8414, 284.87698, 286.72708, 303.21405, 302.3201, 293.57004, 319.35092, 237.71196, 184.1433]
2Float32[329.78833, 230.77928, 325.4293, 372.52838, 333.68506, 355.2007, 228.98228, 251.99353, 364.22644, 384.64645, 451.16382, 418.54965, 467.9811, 445.20676, 336.06516, 289.43478, 346.6008, 325.68512, 416.63342, 516.20215, 391.5713, 442.821, 279.20056, 299.8414, 284.87698, 286.72708, 303.21405, 302.3201, 293.57004, 319.35092, 237.71196, 184.1433]


32-element Array{Float32,1}:
 329.78833
 230.77928
 325.4293
 372.52838
 333.68506
 355.2007
 228.98228
 251.99353
 364.22644
 384.64645
 451.16382
 418.54965
 467.9811
   ⋮
 391.5713
 442.821
 279.20056
 299.8414
 284.87698
 286.72708
 303.21405
 302.3201
 293.57004
 319.35092
 237.71196
 184.1433

In [21]:
import Random # hide
Random.seed!(1729) # hide
osize = XIO3(4, 5, 3)
isizes = [XIO3(3, 4, 3), XIO3(5, 3, 2)]
e = SparseFocalEncoder(isizes, osize, [1, 1])

SparseFocalEncoder(Array{Float32,1}[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], FocalMatrix[FocalMatrix(XIO3[3×4×3], XIO3[4×5×3], 1, UInt16[1 2; 1 3; 2 3; 2 3], UInt16[1 2; 1 3; … ; 2 4; 3 4], UInt32[1 37 … 145 199; 235 289 … 451 532; 586 622 … 730 784; 820 856 … 964 1018], Float32[0.9767926, 0.4211475, 0.8915801, 0.19209142, 0.3656292, 0.25811878, 0.16790241, 0.1177921, 0.43690702, 0.22150604  …  0.3735941, 0.88273233, 0.3112477, 0.7634302, 0.4221577, 0.3559206, 0.15888774, 0.76714975, 0.98783547, 0.16438827], 1054), FocalMatrix(XIO3[5×3×2], XIO3[4×5×3], 1, UInt16[1 2; 1 3; 3 5; 4 5], UInt16[1 2; 1 3; … ; 2 3; 2 3], UInt32[1 25 … 97 121; 145 181 … 289 325; 361 397 … 505 541; 577 601 … 673 697], Float32[0.501426

In [22]:
x, tops = sparse(osize, rand_array(AW, osize))

(Float32[0.9825994, 0.0, 0.0, 0.0, 0.0, 0.09322806, 0.11758536, 0.0, 0.0, 0.7573908  …  0.0, 0.1951818, 0.0, 0.0, 0.591828, 0.0, 0.0, 0.0, -0.09235107, 0.0], UInt16[1, 3, 1, 1, 3, 1, 2, 1, 3, 3, 3, 2, 2, 1, 1, 3, 1, 1, 1, 2])

In [23]:
function decode!(m::FocalMatrix, ins::Vector{AW}, outs::Vector{AW}) 
    idxW = 1
    local diz = m.isize.z
    local dix = 1; local diy = 1; local o = 0.0
    osize = m.osize
    isize = m.isize
    ws = m.ws
    xs, ys = m.xs, m.ys
    local bz = isize.z
    for ox::XIO in 1:osize.x
        ax, bx = xs[ox, :]
        dix = 1 + bx - ax
        for oy::XIO in 1:osize.y
            ay, by = ys[oy, :]
            diy = 1 + by - ay
            for oz::XIO in 1:osize.z
                oPos = oz + osize.z * (oy - 1) + (osize.z * osize.y) * (ox - 1)
                o = outs[oPos]
                for ix::XIO in ax:bx, iy::XIO in ay:by
                    iPos = isize.z * (iy - 1) + (isize.z * isize.y) * (ix - 1)
                    for iz::XIO in 1:bz
                        ins[iPos + iz] = ins[iPos + iz] + o * ws[idxW + iz]
                    end
                    idxW += diz
                end
            end
        end
    end
    ins
end

decode! (generic function with 1 method)

In [24]:
prediction = [zero_array(AW, isize) for isize in isizes]
length.(prediction)

2-element Array{Int64,1}:
 36
 30

In [25]:
Random.seed!(1776)
dense_outs = rand_array(AW, osize)

60-element Array{Float32,1}:
 -0.5040101
 -0.44363344
 -0.581382
 -0.47755456
  0.7569803
 -0.95768476
  0.19346286
  0.5086846
 -0.58245766
  0.114865154
  0.8170378
  0.3969726
  0.7598583
  ⋮
 -0.81163275
 -0.45565706
  0.2914844
 -0.21604145
 -0.77984816
  0.50165206
  0.5998679
 -0.006974394
  0.12582617
 -0.7443067
  0.50543666
  0.34178364

In [26]:
sparse_outs, tops = sparse(osize, dense_outs)

(Float32[0.0, -0.44363344, 0.0, 0.0, 0.7569803, 0.0, 0.0, 0.5086846, 0.0, 0.0  …  0.2914844, 0.0, 0.0, 0.50165206, 0.5998679, 0.0, 0.0, 0.0, 0.50543666, 0.0], UInt16[2, 2, 2, 2, 1, 1, 3, 1, 3, 1, 1, 1, 2, 2, 1, 1, 3, 3, 1, 2])

In [27]:
decode!(e.weight_tensors[1], prediction[1], sparse_outs)

36-element Array{Float32,1}:
 1.5002005
 1.3328108
 0.7865439
 1.3904957
 0.9960151
 0.9816769
 2.7919016
 2.2018158
 2.7635148
 2.2775857
 1.578128
 2.420969
 2.8091235
 ⋮
 2.0054297
 2.3129823
 1.4821259
 2.9570692
 3.4465995
 3.2989042
 3.5922203
 3.0865903
 3.777996
 1.4576781
 2.0125113
 2.2678413

In [28]:
sparse_pred, tops = sparse(isizes[1], prediction[1])

(Float32[1.5002005, 0.0, 0.0, 1.3904957, 0.0, 0.0, 2.7919016, 0.0, 0.0, 0.0  …  0.0, 0.0, 3.4465995, 0.0, 0.0, 0.0, 3.777996, 0.0, 0.0, 2.2678413], UInt16[1, 1, 1, 3, 1, 3, 3, 1, 2, 2, 3, 3])

In [29]:
pred = copy(sparse_pred)
pred_forward = zero_array(AW, osize)
forward!(e.weight_tensors[1], pred, pred_forward)

60-element Array{Float32,1}:
  2.112439
  5.1901646
  4.0187054
  9.694454
  9.226432
  7.463846
 12.683958
 12.4511175
 11.755856
 11.784513
 16.878277
  9.072924
  6.4432597
  ⋮
 16.423084
  7.9073987
 12.055057
 14.616652
 11.608127
  8.555617
 19.157906
 10.970013
 14.31929
  6.8720512
  8.644271
  6.9648275

In [30]:
pred_forward - dense_outs

60-element Array{Float32,1}:
  2.6164489
  5.633798
  4.600087
 10.1720085
  8.469452
  8.421531
 12.490495
 11.942433
 12.338313
 11.669648
 16.06124
  8.675951
  5.6834016
  ⋮
 17.234716
  8.363056
 11.763572
 14.832693
 12.387975
  8.053966
 18.558037
 10.976987
 14.193464
  7.616358
  8.138834
  6.623044

In [31]:
Random.seed!(1234)

is = XIO3(8,10,3)
os = XIO3(5,4,3)
r = 3
f = FocalMatrix(is, os, r)

FocalMatrix(XIO3[8×10×3], XIO3[5×4×3], 3, UInt16[1 4; 1 6; … ; 3 8; 4 8], UInt16[1 4; 1 7; 3 9; 6 10], UInt32[1 145 397 649; 829 1045 1423 1801; … ; 3520 3736 4114 4492; 4762 4942 5257 5572], Float32[0.59084463, 0.76679707, 0.5662374, 0.46008533, 0.7940257, 0.8541466, 0.20058604, 0.29861426, 0.24683718, 0.5796722  …  0.708213, 0.73846525, 0.9270241, 0.4111475, 0.48356026, 0.84817374, 0.11161669, 0.7702865, 0.6142066, 0.3792697], 5797)

In [32]:
import OgmaJL.forward!
function forward!(m::FocalMatrix, ins::Array{AW}, outs::Array{AW})
    idxW = 1
    local diz = m.isize.z
    local dix = 1; local diy = 1; local o = 0.0
    osize = m.osize
    isize = m.isize
    ws = m.ws
    xs, ys = m.xs, m.ys
    local bz = isize.z
    for ox::XIO in 1:osize.x
        ax, bx = xs[ox, :]
        dix = 1 + bx - ax
        for oy::XIO in 1:osize.y
            ay, by = ys[oy, :]
            diy = 1 + by - ay
            for oz::XIO in 1:osize.z
                o = 0.0
                for ix::XIO in ax:bx, iy::XIO in ay:by
                    iPos = isize.z * (iy - 1) + (isize.z * isize.y) * (ix - 1)
                    for iz::XIO in 1:bz
                        o = o + ins[iPos + iz] * ws[idxW + iz]
                    end
                    idxW += diz
                end
                oPos = oz + osize.z * (oy - 1) + (osize.z * osize.y) * (ox - 1)
                outs[oPos] = o
            end
        end
    end
    outs
end


forward! (generic function with 6 methods)

In [33]:
f.xs == [1 4; 1 6; 1 7; 3 8; 4 8] && 
f.ys == [1 4; 1 7; 3 9; 6 10] &&
f.ws[1:10] ≈ [0.59084463, 0.76679707, 0.5662374, 0.46008533, 0.7940257,
    0.8541466, 0.20058604, 0.29861426, 0.24683718, 0.5796722]

true

In [34]:
Random.seed!(56789)
ins = AW.(rand([-1,1], is.x * is.y * is.z) .* rand(is.x * is.y * is.z))
outs = zeros(AW, os.x * os.y * os.z)
forward!(f, ins, outs)

60-element Array{Float32,1}:
  2.369688
  1.9627601
  1.6924155
  5.680007
  5.5550647
  4.608256
  5.510908
  5.391149
  2.8863041
  1.8599162
  2.4497209
  1.8094441
 -0.5646462
  ⋮
 -0.98360896
 -2.557888
 -0.024428785
  0.91814095
 -2.7633836
 -0.1305455
  0.9920449
 -6.7878976
  0.48425412
 -3.4415362
 -0.012607932
 -1.3955071

In [35]:
println(outs)

Float32[2.369688, 1.9627601, 1.6924155, 5.680007, 5.5550647, 4.608256, 5.510908, 5.391149, 2.8863041, 1.8599162, 2.4497209, 1.8094441, -0.5646462, 2.496545, 2.7496731, 4.4924946, 1.7888093, 4.1376715, 3.7172203, 1.2116854, 0.7597302, -2.3244824, -4.2741694, -1.955646, 1.8763541, -0.37737405, 3.562166, -1.4183358, 1.867738, 2.8685708, -1.6369025, -0.32065183, -0.61710286, -1.6649387, -2.9539576, -1.0418377, -0.27939826, 0.30634487, -0.7195828, -2.1637728, -2.7929268, -0.86904335, 3.0336506, 1.4393296, -2.7035058, 0.7345776, 1.9890188, -1.8297501, -0.98360896, -2.557888, -0.024428785, 0.91814095, -2.7633836, -0.1305455, 0.9920449, -6.7878976, 0.48425412, -3.4415362, -0.012607932, -1.3955071]


In [36]:
outs ≈ [2.3696885, 1.9627601, 1.6924154, 5.6800075, 5.5550666, 4.608255, 5.5109096, 5.3911486, 2.8863041, 1.8599161, 2.449721, 1.8094438, -0.56464577, 2.4965448, 2.7496734, 4.4924912, 1.7888112, 4.13767, 3.7172208, 1.2116852, 0.7597295, -2.3244824, -4.274169, -1.9556457, 1.8763535, -0.3773745, 3.562166, -1.4183352, 1.8677393, 2.8685718, -1.6369021, -0.3206531, -0.6171037, -1.6649392, -2.9539573, -1.0418377, -0.2793983, 0.30634505, -0.7195828, -2.1637726, -2.792926, -0.8690437, 3.03365, 1.4393299, -2.7035058, 0.73457795, 1.9890194, -1.8297504, -0.9836089, -2.5578885, -0.024428671, 0.9181401, -2.7633839, -0.13054581, 0.9920448, -6.787898, 0.484254, -3.441536, -0.012607433, -1.3955071]
    

true

In [37]:
sparseouts, tops = sparse(f, outs)

(Float32[2.369688, 0.0, 0.0, 5.680007, 0.0, 0.0, 5.510908, 0.0, 0.0, 0.0  …  -0.024428785, 0.91814095, 0.0, 0.0, 0.9920449, 0.0, 0.0, 0.0, -0.012607932, 0.0], UInt16[1, 1, 1, 2, 3, 1, 1, 3, 3, 3, 2, 3, 2, 3, 1, 2, 3, 1, 1, 2])

In [38]:
println(sparseouts)
println(tops)

Float32[2.369688, 0.0, 0.0, 5.680007, 0.0, 0.0, 5.510908, 0.0, 0.0, 0.0, 2.4497209, 0.0, 0.0, 0.0, 2.7496731, 4.4924946, 0.0, 0.0, 3.7172203, 0.0, 0.0, 0.0, 0.0, -1.955646, 0.0, 0.0, 3.562166, 0.0, 0.0, 2.8685708, 0.0, -0.32065183, 0.0, 0.0, 0.0, -1.0418377, 0.0, 0.30634487, 0.0, 0.0, 0.0, -0.86904335, 3.0336506, 0.0, 0.0, 0.0, 1.9890188, 0.0, 0.0, 0.0, -0.024428785, 0.91814095, 0.0, 0.0, 0.9920449, 0.0, 0.0, 0.0, -0.012607932, 0.0]
UInt16[1, 1, 1, 2, 3, 1, 1, 3, 3, 3, 2, 3, 2, 3, 1, 2, 3, 1, 1, 2]


In [39]:
Random.seed!(34567)
osize = XIO3(4, 5, 3)
isizes = [XIO3(3, 4, 3), XIO3(5, 3, 2)]
e = OgmaJL.SparseFocalEncoder(isizes, osize, [1, 1])

Random.seed!(89123)
outs = zero_array(AW, osize)
# outs = -1 .+ (2 .* rand(length(outs)))
# ins = [(-1 .+ AW.(2 .* rand(isize.x * isize.y * isize.z))) for isize in isizes]
ins = [rand_array(AW, isize) for isize in isizes]
e.inputs .= ins
e.outputs .= outs
forward!(e)

1Float32[0.30973428, 0.5089234, -0.2817986, -0.8455551, -1.2917488, -0.14803626, 0.5260522, -0.45357493, 0.14881156, -0.7219375, -1.1244273, -0.51630723, -0.70497763, -0.7184298, -0.4166844, -0.004550904, -1.1152643, -1.0538586, -1.0178995, -1.9452542, -1.925517, -0.2472769, -1.0106049, -0.40791374, -1.450501, -1.266211, 0.72933006, -0.22153974, -0.87707627, -1.8283428, 0.43614906, 0.4975979, 0.4317258, -1.170793, -1.424866, -0.756474, 0.1219134, -1.4473423, -0.7918875, -1.098767, -0.08688003, -1.2527707, -0.91790813, -1.5168632, -0.20279375, -0.66519207, -0.39083898, 0.9330815, -0.61802274, 0.20777959, -1.195794, -0.54686576, 0.524223, -0.29333836, -1.3987383, -1.1674423, -1.5050104, -0.3903559, -0.6258691, -0.20152543]
2Float32[-0.17616624, 0.19611296, -0.01586762, 0.52496195, -1.4770164, -0.23483497, 0.08747473, -0.40441868, 1.3073246, -1.1927086, -1.4557223, -0.012100697, 0.44500488, -1.5216341, -0.3198055, 0.8358241, -1.8722794, 0.38112888, 0.7258484, -2.8219247, -0.97797686, -0.2

60-element Array{Float32,1}:
 -0.17616624
  0.19611296
 -0.01586762
  0.52496195
 -1.4770164
 -0.23483497
  0.08747473
 -0.40441868
  1.3073246
 -1.1927086
 -1.4557223
 -0.012100697
  0.44500488
  ⋮
 -0.9082285
  0.010027856
 -1.8122463
 -1.4337664
  0.22968581
 -0.33851877
 -1.0805783
 -1.5254555
 -1.1883162
 -0.65380657
 -0.882862
 -0.31242907

In [40]:
println(e.outputs)

Float32[-0.17616624, 0.19611296, -0.01586762, 0.52496195, -1.4770164, -0.23483497, 0.08747473, -0.40441868, 1.3073246, -1.1927086, -1.4557223, -0.012100697, 0.44500488, -1.5216341, -0.3198055, 0.8358241, -1.8722794, 0.38112888, 0.7258484, -2.8219247, -0.97797686, -0.26038912, -0.6596564, -0.09790245, -2.9023488, -2.7865653, 0.33717537, 0.68588126, -2.0352392, -4.1352415, 1.5837169, 1.217061, 1.2314525, -2.0370488, -2.9956234, -0.4740031, -0.9301128, -0.69790065, -1.1098443, -2.004672, -0.988083, -1.4327043, -2.0561285, -2.701246, -1.883915, -0.92355955, -0.06510103, 1.3771538, -0.9082285, 0.010027856, -1.8122463, -1.4337664, 0.22968581, -0.33851877, -1.0805783, -1.5254555, -1.1883162, -0.65380657, -0.882862, -0.31242907]


In [41]:
sparseouts, tops = sparse(osize, e.outputs)

(Float32[0.0, 0.19611296, 0.0, 0.52496195, 0.0, 0.0, 0.0, 0.0, 1.3073246, 0.0  …  0.0, 0.0, 0.22968581, 0.0, -1.0805783, 0.0, 0.0, 0.0, 0.0, -0.31242907], UInt16[2, 1, 3, 3, 1, 1, 1, 3, 3, 1, 1, 3, 2, 2, 3, 3, 2, 2, 1, 3])

In [42]:
println(sparseouts)
println(tops)

Float32[0.0, 0.19611296, 0.0, 0.52496195, 0.0, 0.0, 0.0, 0.0, 1.3073246, 0.0, 0.0, -0.012100697, 0.44500488, 0.0, 0.0, 0.8358241, 0.0, 0.0, 0.7258484, 0.0, 0.0, 0.0, 0.0, -0.09790245, 0.0, 0.0, 0.33717537, 0.68588126, 0.0, 0.0, 1.5837169, 0.0, 0.0, 0.0, 0.0, -0.4740031, 0.0, -0.69790065, 0.0, 0.0, -0.988083, 0.0, 0.0, 0.0, -1.883915, 0.0, 0.0, 1.3771538, 0.0, 0.010027856, 0.0, 0.0, 0.22968581, 0.0, -1.0805783, 0.0, 0.0, 0.0, 0.0, -0.31242907]
UInt16[2, 1, 3, 3, 1, 1, 1, 3, 3, 1, 1, 3, 2, 2, 3, 3, 2, 2, 1, 3]


In [43]:
Random.seed!(34567)
osize = XIO3(4, 5, 3)
isizes = [XIO3(3, 4, 3), XIO3(5, 3, 2)]
e = OgmaJL.SparseFocalEncoder(isizes, osize, [1, 1])

Random.seed!(89123)
outs = zero_array(AW, osize)
# outs = -1 .+ (2 .* rand(length(outs)))
# ins = [(-1 .+ AW.(2 .* rand(isize.x * isize.y * isize.z))) for isize in isizes]
ins = [rand_array(AW, isize) for isize in isizes]
e.inputs .= ins
e.outputs .= outs
forward!(e)
println(e.outputs)

1Float32[0.30973428, 0.5089234, -0.2817986, -0.8455551, -1.2917488, -0.14803626, 0.5260522, -0.45357493, 0.14881156, -0.7219375, -1.1244273, -0.51630723, -0.70497763, -0.7184298, -0.4166844, -0.004550904, -1.1152643, -1.0538586, -1.0178995, -1.9452542, -1.925517, -0.2472769, -1.0106049, -0.40791374, -1.450501, -1.266211, 0.72933006, -0.22153974, -0.87707627, -1.8283428, 0.43614906, 0.4975979, 0.4317258, -1.170793, -1.424866, -0.756474, 0.1219134, -1.4473423, -0.7918875, -1.098767, -0.08688003, -1.2527707, -0.91790813, -1.5168632, -0.20279375, -0.66519207, -0.39083898, 0.9330815, -0.61802274, 0.20777959, -1.195794, -0.54686576, 0.524223, -0.29333836, -1.3987383, -1.1674423, -1.5050104, -0.3903559, -0.6258691, -0.20152543]
2Float32[-0.17616624, 0.19611296, -0.01586762, 0.52496195, -1.4770164, -0.23483497, 0.08747473, -0.40441868, 1.3073246, -1.1927086, -1.4557223, -0.012100697, 0.44500488, -1.5216341, -0.3198055, 0.8358241, -1.8722794, 0.38112888, 0.7258484, -2.8219247, -0.97797686, -0.2

In [44]:
sparseouts, tops = sparse(osize, e.outputs)
println(sparseouts)
println(tops)

Float32[0.0, 0.19611296, 0.0, 0.52496195, 0.0, 0.0, 0.0, 0.0, 1.3073246, 0.0, 0.0, -0.012100697, 0.44500488, 0.0, 0.0, 0.8358241, 0.0, 0.0, 0.7258484, 0.0, 0.0, 0.0, 0.0, -0.09790245, 0.0, 0.0, 0.33717537, 0.68588126, 0.0, 0.0, 1.5837169, 0.0, 0.0, 0.0, 0.0, -0.4740031, 0.0, -0.69790065, 0.0, 0.0, -0.988083, 0.0, 0.0, 0.0, -1.883915, 0.0, 0.0, 1.3771538, 0.0, 0.010027856, 0.0, 0.0, 0.22968581, 0.0, -1.0805783, 0.0, 0.0, 0.0, 0.0, -0.31242907]
UInt16[2, 1, 3, 3, 1, 1, 1, 3, 3, 1, 1, 3, 2, 2, 3, 3, 2, 2, 1, 3]
